In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
# batch_size = 4
batch_size = 64

In [3]:
from keras.datasets import mnist

In [10]:
(X_train,y_train),(X_test,y_test) = mnist.load_data()

In [11]:
(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [12]:
X_train = np.expand_dims(X_train,1)
X_test = np.expand_dims(X_test,1)

In [13]:
X_train.shape,X_train.shape

((60000, 1, 28, 28), (60000, 1, 28, 28))

In [14]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [15]:
y_train = onehot(y_train)
y_test = onehot(y_test)

In [16]:
y_train[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [23]:
mean = X_train.mean().astype(np.float32)
std_dev = X_train.std().astype(np.float32)
X_test.mean()

33.791224489795916

In [24]:
def normalize(x):
    return (x - mean)/std_dev

In [25]:
def get_linear_model():
    model = Sequential([
        Lambda(normalize,input_shape=(1,28,28)),
        Flatten(),
        Dense(10,activation='softmax')
    ])
    model.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [26]:
lm = get_linear_model()

In [27]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train,y_train,batch_size=batch_size)
test_batches = gen.flow(X_test,y_test,batch_size=batch_size)

In [29]:
lm.fit_generator(batches,batches.N,nb_epoch=1,
                validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 11s - loss: 0.4138 - acc: 0.8768 - val_loss: 0.3021 - val_acc: 0.9137


In [30]:
lm.optimizer.lr = 0.1

In [32]:
lm.fit_generator(batches,batches.N,nb_epoch=1,
                validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/1
60000/60000 [==============================] - 11s - loss: 0.2985 - acc: 0.9148 - val_loss: 0.2864 - val_acc: 0.9185


In [33]:
lm.optimizer.lr = 0.01

In [36]:
lm.fit_generator(batches,batches.N,nb_epoch=3,
                validation_data=test_batches,nb_val_samples=test_batches.N)

Epoch 1/3
60000/60000 [==============================] - 11s - loss: 0.2838 - acc: 0.9206 - val_loss: 0.2813 - val_acc: 0.9212
Epoch 2/3
60000/60000 [==============================] - 11s - loss: 0.2768 - acc: 0.9230 - val_loss: 0.2825 - val_acc: 0.9206
Epoch 3/3
60000/60000 [==============================] - 11s - loss: 0.2722 - acc: 0.9233 - val_loss: 0.2812 - val_acc: 0.9214


In [37]:
# lm.save_weights('../data/mnist/models/lm_0.01.h5')

In [38]:
lm.load_weights('../data/mnist/models/lm_0.01.h5')